- We just saw that Python counts the number of references to an object stored in memory
    - When the reference count goes down to zero, the object is deleted and the memory address is made available to store new objects

___

# What are circular references?

### Example 1

- As we've seen so far, let's say we have a variable `my_var` that points to an object in memory
    - The reference count for the object is equal to 1 (only `my_var` points to it)
- Next, we redefine `my_var` to be equal to `None`
    - This causes the reference count for the object in memory to go to zero
        - Python deletes the object, and we're all good
        
### Example 2

- This time, we again have `my_var` pointing to the object in memory, but **the object contains a reference to another object**
    - When we redefine `my_var` to be `None` the reference count for Object 1 decreases to zero, but the reference count for Object 2 stays at 1 (since Object 1 still references it)
        - Since Object 1 has reference count zero, it gets destroyed
            - This causes the reference count for Object 2 to go to zero, and it is also destroyed
                - Again, we're all good
                
### Example 3

- Now, consider the same scenario as Example 2, except not only does Object 1 reference Object 2, but Object 2 also references Object 1
    - Before `my_var` is set to be `None`, the reference count for Object 1 is 2 (one from `my_var`, and one from Object 2)
        - Therefore, when we redefine `my_var` to be `None`, the reference count for Object 1 decreases to 1
            - ***But this means Object 1 and 2 will still have a reference count of 1***
                - ***THEY WON'T BE DELETED***
                    - This scenario is called a **circular reference**
- Since the Python memory management system works off reference counts, these objects would never be deleted
    - This causes a **memory leak**

____

# What is the garbage collector?

- The garbage collector removes these circular references
    - Eliminates memory leaks
- We can interact with the garbage collector through the `gc` module
    - By default, garbage collection is enabled
        - We can manually turn it off, but it's dangerous
- *Why would we want to turn off the garbage collection anyway?*
    - In advanced cases, we may want to optimize our program
        - Garbage collection uses up some processing power to run
- In general, garbage collection works fine
    - In older versions of python (<3.4), there could be some issues

____

# Example

In [1]:
import ctypes
import gc

In [2]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [3]:
def object_by_id(object_id):
    for obj in gc.get_objects():
        if id(obj) == object_id:
            return "Object exists"
    return "Not found"

- Defining classes with circular references

In [4]:
class A:
    def __init__(self):
        self.b = B(self)
        print(f'A: self:{hex(id(self))}, b:{hex(id(self.b))}')

In [5]:
class B:
    def __init__(self, a):
        self.a = a
        print(f'B: self:{hex(id(self))}, a:{hex(id(self.a))}')

- Disabling garbage collection

In [6]:
my_var = A()

B: self:0x220583fc860, a:0x220583fc828
A: self:0x220583fc828, b:0x220583fc860


- As we can see, the memory address of a in B is the same as the memory address of A (and vice-versa)

In [7]:
hex(id(my_var))

'0x220583fc828'

- Also, the memory address of `my_var` is the same as A

In [8]:
hex(id(my_var.b))

'0x220583fc860'

In [9]:
hex(id(my_var.b.a))

'0x220583fc828'

- We're now satisfied that we have a circular reference

In [10]:
a_id = id(my_var)
b_id = id(my_var.b)

In [11]:
print(ref_count(a_id),ref_count(b_id))

2 1


- As expected, the address for which `my_var` is an alias (i.e. the object A) has two references
    - One from `my_var`, and one from Object B

- Now, we'll check whether these objects (A and B) are being tracked by the garbage collector

In [12]:
object_by_id(a_id)

'Object exists'

In [13]:
object_by_id(b_id)

'Object exists'

- Now, we destroy the reference from `my_var`

In [14]:
my_var = None

In [15]:
ref_count(a_id)

1

- Now that we've removed the reference from `my_var`, we're only left with the circular reference
    - *Are the objects still being tracked by the garbage collector?*

In [16]:
object_by_id(a_id),object_by_id(b_id)

('Object exists', 'Object exists')

- Yup
    - We can manually run the garbage collector

In [17]:
gc.collect()

39

In [18]:
object_by_id(a_id),object_by_id(b_id)

('Not found', 'Not found')

- Neither of them are found!
    - They've been removed!